<a href="https://colab.research.google.com/github/xbetik/DiusAIInvoiceExtractionApp/blob/master/DiusAI_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install "transformers>=4.44" "accelerate>=0.33" "huggingface_hub>=0.23" \
             "sentencepiece>=0.1.99" "protobuf>=3.20.2,<5" \
             "Pillow>=10.2" "pdf2image>=1.17" "safetensors>=0.4.2"

Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (1,013 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126441 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import os, json
from typing import Dict, Any
from PIL import Image
from transformers import pipeline

def normalize_amount(x: str) -> float | None:
    if not x: return None
    s = x.replace("\u00A0", " ").strip().replace(" ", "")
    if "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "").replace(",", ".")
        else:
            s = s.replace(",", "")
    elif "," in s:
        s = s.replace(",", ".")
    try:
        return float(s.strip("EUR€$£CZKPLNCHFHUF"))
    except:
        return None

def normalize_record(record: Dict[str, Any]) -> Dict[str, Any]:
    supplier = record.get("supplier", {}) or {}
    customer = record.get("customer", {}) or {}
    out = {
        "document_id": record.get("document_id"),
        "source_type": record.get("source_type"),
        "supplier": {
            "name": supplier.get("name"),
            "vat_id": supplier.get("vat_id"),
            "ico": supplier.get("ico"),
            "address": supplier.get("address"),
            "iban": supplier.get("iban"),
        },
        "customer": {
            "name": customer.get("name"),
            "vat_id": customer.get("vat_id"),
            "ico": customer.get("ico"),
            "address": customer.get("address"),
        },
        "invoice_number": record.get("invoice_number"),
        "issue_date": record.get("issue_date"),
        "due_date": record.get("due_date"),
        "delivery_date": record.get("delivery_date"),
        "currency": record.get("currency") or "EUR",
        "total_amount": normalize_amount(record.get("total_amount")),
        "vat_amount": normalize_amount(record.get("vat_amount")),
        "subtotal_amount": normalize_amount(record.get("subtotal_amount")),
        "payment_method": record.get("payment_method"),
        "lines": record.get("lines") if isinstance(record.get("lines"), list) else [],
        "notes": record.get("notes"),
        "confidence": float(record.get("confidence") or 0.0),
        "missing_fields": [],
        "status": "incomplete"
    }
    missing = []
    if not out["supplier"].get("name"): missing.append("supplier.name")
    if not out["customer"].get("name"): missing.append("customer.name")
    if not out["invoice_number"]: missing.append("invoice_number")
    if not out["issue_date"]: missing.append("issue_date")
    if out["total_amount"] in (None, "", []): missing.append("total_amount")
    out["missing_fields"] = missing
    out["status"] = "complete" if not missing else "incomplete"
    if not record.get("confidence"):
        out["confidence"] = max(0.3, 0.95 - 0.05*len(missing))
    return out

In [ ]:
docvqa = pipeline(
    "document-question-answering",
    model="naver-clova-ix/donut-base-finetuned-docvqa",
    tokenizer="naver-clova-ix/donut-base-finetuned-docvqa",
    device_map="auto"   # GPU if available
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/803M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

...

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu


In [ ]:
from google.colab import files
uploaded = files.upload()

invoice_path = list(uploaded.keys())[0]  # take first uploaded file
print("Uploaded:", invoice_path)

# If PDF, take the first page
if invoice_path.lower().endswith(".pdf"):
    from pdf2image import convert_from_path
    pages = convert_from_path(invoice_path, dpi=200)
    image = pages[0]
else:
    image = Image.open(invoice_path).convert("RGB")

In [ ]:
def ask(question: str) -> str:
    res = docvqa(image=image, question=question)
    if isinstance(res, list) and res:
        return res[0].get("answer", "")
    return res.get("answer", "")

record = {
    "document_id": os.path.basename(invoice_path),
    "source_type": "pdf" if invoice_path.lower().endswith(".pdf") else "image",
    "supplier": {
        "name": ask("What is the supplier name?"),
        "address": ask("What is the supplier address?"),
    },
    "customer": {
        "name": ask("What is the customer name?"),
        "address": ask("What is the customer address?"),
    },
    "vat_amount": ask("What is the VAT amount?"),
    "total_amount": ask("What is the total amount to pay?"),
    "lines": [],
    "notes": None,
    "confidence": 0.0,
}

normalized = normalize_record(record)

`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.


In [ ]:
with open("invoice_ml.json", "w", encoding="utf-8") as f:
    json.dump(normalized, f, indent=2, ensure_ascii=False)

print(json.dumps(normalized, indent=2, ensure_ascii=False))
files.download("invoice_ml.json")

{
  "document_id": "Invoice (1).pdf",
  "source_type": "pdf",
  "supplier": {
    "name": "$0.00",
    "vat_id": null,
    "ico": null,
    "address": "$0.00",
    "iban": null
  },
  "customer": {
    "name": "will smith",
    "vat_id": null,
    "ico": null,
    "address": "$120.00"
  },
  "invoice_number": null,
  "issue_date": null,
  "due_date": null,
  "delivery_date": null,
  "currency": "EUR",
  "total_amount": 0.0,
  "vat_amount": 0.0,
  "subtotal_amount": null,
  "payment_method": null,
  "lines": [],
  "notes": null,
  "confidence": 0.85,
  "missing_fields": [
    "invoice_number",
    "issue_date"
  ],
  "status": "incomplete"
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>